In [1]:
import subprocess
import os
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, lower, regexp_replace, count
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.functions import lit
import tensorflow as tf
import numpy as np
from keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

def run_command(command):
    """Utility function to run shell commands"""
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    if process.returncode == 0:
        print(f"Success: {command}\nOutput:\n{stdout.decode()}")
    else:
        print(f"Error: {command}\nError Message:\n{stderr.decode()}")

# Restart DFS
print("Stopping DFS...")
run_command("stop-dfs.sh")

print("Starting DFS...")
run_command("start-dfs.sh")

# Check running Java services
print("Checking Java processes...")
run_command("jps")

# Check Safe Mode status
print("Checking Safe Mode status...")
run_command("hdfs dfsadmin -safemode get")

# Leave Safe Mode if it's ON
print("Leaving Safe Mode if necessary...")
run_command("hdfs dfsadmin -safemode leave")

# Create HDFS directories
hdfs_dirs = [
    "e-commerce/datasets",
    "e-commerce/splits",
    "e-commerce/models",
    "e-commerce/outputs"
]

for hdfs_dir in hdfs_dirs:
    print(f"Creating HDFS directory: {hdfs_dir}")
    run_command(f"hdfs dfs -mkdir -p {hdfs_dir}")

print("All tasks completed successfully!")

# Download dataset from Kaggle
kaggle_dataset_path = "~/kaggle-datasets"
dataset_name = "indonesia-top-ecommerce-unicorn-tweets"
print("Downloading dataset from Kaggle...")
run_command(f"kaggle datasets download -d robertvici/{dataset_name} -p {kaggle_dataset_path}")

# Unzip the downloaded dataset
zip_file_path = f"{kaggle_dataset_path}/{dataset_name}.zip"
print("Unzipping dataset...")
run_command(f"unzip -o {zip_file_path} -d {kaggle_dataset_path}")

# Upload files to HDFS
print("Uploading JSON files to HDFS...")
run_command(f"hdfs dfs -put {kaggle_dataset_path}/*.json e-commerce/datasets/")


# Initialize Spark Session
spark = SparkSession.builder \
    .appName("E-Commerce Engagement Prediction ML") \
    .getOrCreate()

# Load datasets
blibli_df = spark.read.json('e-commerce/datasets/bliblidotcom.json')
bukalapak_df = spark.read.json('e-commerce/datasets/bukalapak.json')
lazadaID_df = spark.read.json('e-commerce/datasets/lazadaID.json')
shopeeID_df = spark.read.json('e-commerce/datasets/ShopeeID.json')
tokopedia_df = spark.read.json('e-commerce/datasets/tokopedia.json')

# Add a new column to identify the company source
blibli_df = blibli_df.withColumn('source', lit('blibli'))
bukalapak_df = bukalapak_df.withColumn('source', lit('bukalapak'))
lazadaID_df = lazadaID_df.withColumn('source', lit('lazadaID'))
shopeeID_df = shopeeID_df.withColumn('source', lit('shopeeID'))
tokopedia_df = tokopedia_df.withColumn('source', lit('tokopedia'))

# Merge datasets using union (axis=0 equivalent in Spark)
merged_df = blibli_df.union(bukalapak_df).union(lazadaID_df).union(shopeeID_df).union(tokopedia_df)

# Clean tweet text
def clean_text(text):
    return text.lower().replace("#", "").strip()

clean_text_udf = udf(clean_text, StringType())

# Apply text cleaning and create new features
data_cleaned = merged_df.withColumn("clean_tweet", clean_text_udf(col("tweet"))) \
                       .withColumn("engagement", col("replies_count") + col("retweets_count") + col("likes_count"))

# Select relevant features
selected_data = data_cleaned.select(
    col("clean_tweet").alias("text"),
    col("replies_count").alias("replies"),
    col("retweets_count").alias("retweets"),
    col("likes_count").alias("likes"),
    col("engagement").alias("target"),
    col("hashtags"),    
    col("source")
)

# Split dataset
train_data, validate_data, test_data = selected_data.randomSplit([0.7, 0.15, 0.15], seed=42)

# Save splits for later use
train_data.write.json("e-commerce/splits/train.json", mode="overwrite")
validate_data.write.json("commerce/splits/validate.json", mode="overwrite")
test_data.write.json("commerce/splits/test.json", mode="overwrite")

# Change null value with 0 (if any)
merged_df = merged_df.fillna({"likes_count": 0, "replies_count": 0, "retweets_count": 0})

from pyspark.sql import functions as F

# Check negative value
merged_df.filter((F.col("likes_count") < 0) | (F.col("replies_count") < 0) | (F.col("retweets_count") < 0)).show()

# Change negative value with 0 (if any)
for col in ["likes_count", "replies_count", "retweets_count"]:
    merged_df = merged_df.withColumn(col, F.when(F.col(col) < 0, 0).otherwise(F.col(col)))
    
# Matching target engagement definitions in Spark DataFrame
blibli_df = blibli_df.withColumn("engagement", F.col("likes_count") + F.col("replies_count") + F.col("retweets_count"))
bukalapak_df = bukalapak_df.withColumn("engagement", F.col("likes_count") + F.col("replies_count") + F.col("retweets_count"))
lazadaID_df = lazadaID_df.withColumn("engagement",   F.col("likes_count") + F.col("replies_count") + F.col("retweets_count"))
shopeeID_df = shopeeID_df.withColumn("engagement",   F.col("likes_count") + F.col("replies_count") + F.col("retweets_count"))
tokopedia_df = tokopedia_df.withColumn("engagement", F.col("likes_count") + F.col("replies_count") + F.col("retweets_count"))


# Load train data (convert Spark DataFrame to Pandas)
train_df = train_data.toPandas()

# Tokenize and vectorize text (fit on original text, not the padded sequences)
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(train_df["text"])  # Fit tokenizer on the raw text data

# Convert texts to sequences
X_train = tokenizer.texts_to_sequences(train_df["text"])

# Pad the sequences to ensure uniform length
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, padding='post')

y_train = np.array(train_df["target"])


# Define a simple Neural Network model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=64),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(1, activation="linear")
])

# Compile the model
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

vocab_size = 42500  # Customize with your tokenizer
embedding_dim = 128
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim)

# Example: Tokenize and pad the input text
max_vocab_size = 5000
tokenizer = Tokenizer(num_words=max_vocab_size, oov_token="<UNK>")
tokenizer.fit_on_texts(train_df["text"])  # 'train_df["text"]' should be a list of strings

# Example of saving the tokenizer
with open('tokenizer.pkl', 'wb') as file:
    pickle.dump(tokenizer, file)

X_train_sequences = tokenizer.texts_to_sequences(train_df["text"])
X_train = pad_sequences(X_train_sequences, padding='post')

# Ensure y_train is in the correct format (e.g., a numpy array)
y_train = np.array(train_df["target"])  # Adjust this based on your target column

# Train the model 
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Save the model with a valid file extension in local server
model.save("e-commerce-engagement_model.keras")  # For the native Keras format

# Export to save model
model.export("saved_model/1")
print("Final Model ==> saved_model/1")


Stopping namenodes on [0.0.0.0]
Stopping datanodes
Stopping secondary namenodes [worker01]
Starting namenodes on [0.0.0.0]
Starting datanodes
Starting secondary namenodes [worker01]
214373 DataNode
214555 SecondaryNameNode
212008 SparkSubmit
214680 Jps
214189 NameNode
Safe mode is ON
Safe mode is OFF
Dataset URL: https://www.kaggle.com/datasets/robertvici/indonesia-top-ecommerce-unicorn-tweets
License(s): copyright-authors
indonesia-top-ecommerce-unicorn-tweets.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /home/hduser/kaggle-datasets/indonesia-top-ecommerce-unicorn-tweets.zip
  inflating: /home/hduser/kaggle-datasets/ShopeeID.json  
  inflating: /home/hduser/kaggle-datasets/bliblidotcom.json  
  inflating: /home/hduser/kaggle-datasets/bukalapak.json  
  inflating: /home/hduser/kaggle-datasets/lazadaID.json  
  inflating: /home/hduser/kaggle-datasets/tokopedia.json  
put: `e-commerce/datasets/bliblidotcom.json': File exists
put: `e-com

2024-12-20 20:10:55.202831: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-20 20:10:55.215962: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734700255.231838  213677 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734700255.236460  213677 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-20 20:10:55.252407: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

+--------+---------------+----------+----+---+--------+---+-----------+----+--------+----+----+------+-----+---------+-------------+--------+-------+------------+----------+--------------+------+----+--------+----------+---------+---------+-----+----+-------+-------+----------+--------+-----+
|cashtags|conversation_id|created_at|date|geo|hashtags| id|likes_count|link|mentions|name|near|photos|place|quote_url|replies_count|reply_to|retweet|retweet_date|retweet_id|retweets_count|source|time|timezone|trans_dest|trans_src|translate|tweet|urls|user_id|user_rt|user_rt_id|username|video|
+--------+---------------+----------+----+---+--------+---+-----------+----+--------+----+----+------+-----+---------+-------------+--------+-------+------------+----------+--------------+------+----+--------+----------+---------+---------+-----+----+-------+-------+----------+--------+-----+
+--------+---------------+----------+----+---+--------+---+-----------+----+--------+----+----+------+-----+---------+

2024-12-20 20:11:25.554704: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Epoch 1/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - loss: 845035.1875 - mae: 31.6108
Epoch 2/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 927009.8750 - mae: 33.0268
Epoch 3/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 968650.6250 - mae: 29.3704
Epoch 4/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 853620.5625 - mae: 26.7506
Epoch 5/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 658006.9375 - mae: 26.8828
Epoch 6/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 539886.8125 - mae: 24.9783
Epoch 7/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 514703.7188 - mae: 27.4117
Epoch 8/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 1187194.3750 - mae: 34.2074
Epoch 9/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 918055.8125 - mae: 31.6637
Epoch 10/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: 838902.5000 - mae: 30.5814
INFO:tensorflow:Assets written to: saved_model/1/assets


INFO:tensorflow:Assets written to: saved_model/1/assets


Saved artifact at 'saved_model/1'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 60), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  140520993110736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140520993111312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140520993111888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140520994029840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140520994030992: TensorSpec(shape=(), dtype=tf.resource, name=None)
